In [ ]:
SetS

In [ ]:
install.packages('rpart.plot', dependencies = TRUE, 
                 destdir = "D:/ProgramData/Anaconda2/R/library",
                 repos='http://cran.us.r-project.org',
                 lib = "D:/ProgramData/Anaconda2/R/library",)

install.packages('plotmo', dependencies = TRUE, 
                 destdir = "D:/ProgramData/Anaconda2/R/library",
                 repos='http://cran.us.r-project.org',
                 lib = "D:/ProgramData/Anaconda2/R/library",)

install.packages('lattice', dependencies = TRUE, 
                 destdir = "D:/ProgramData/Anaconda2/R/library",
                 repos='http://cran.us.r-project.org',
                 lib = "D:/ProgramData/Anaconda2/R/library",)

install.packages('rattle', dependencies = TRUE, 
                 destdir = "D:/ProgramData/Anaconda2/R/library",
                 repos='http://cran.us.r-project.org',
                 lib = "D:/ProgramData/Anaconda2/R/library",)

install.packages('RGtk2', dependencies = TRUE, 
                 destdir = "D:/ProgramData/Anaconda2/R/library",
                 repos='http://cran.us.r-project.org',
                 lib = "D:/ProgramData/Anaconda2/R/library",)




title: "Tree.Classificaiton"
output: html_document

In [9]:
library(rpart.plot)
library(rpart)
library(plotmo)
library(lattice)
library(rattle)

Warning message:
"package 'rpart.plot' was built under R version 3.3.3"Loading required package: rpart
Warning message:
"package 'plotmo' was built under R version 3.3.3"Loading required package: plotrix
Warning message:
"package 'plotrix' was built under R version 3.3.3"Loading required package: TeachingDemos
Warning message:
"package 'TeachingDemos' was built under R version 3.3.3"Warning message:
"package 'rattle' was built under R version 3.3.3"

ERROR: Error: package or namespace load failed for 'rattle'


In [ ]:
```{r}
tree.data <- metab_data4[,c(8,12:ncol(metab_data4))]
tree.data <- tree.data %>% filter(Diet %in% c("CD","HF")) %>% select(-one_of("Allyl.isothiocyanate"))
f.rp  <- rpart(Diet ~ ., data=tree.data)
prp(f.rp, type = 3, extra = 3)


## slightly larger than default:
(f.rp2 <- rpart(Diet ~ ., data=tree.data, minsplit = 8, cp = .1))
prp(f.rp2, type=1, extra=3, box.col = f.rp2$y)

(f.rp3 <- rpart(Diet ~ ., data=tree.data, minsplit = 8, cp = .01))
prp(f.rp3, type=1, extra=3,box.col = f.rp3$y)

(f.rp4 <- rpart(Diet ~ ., data=tree.data, minsplit = 8, cp = .001))
prp(f.rp4, type=1, extra=3,box.col = f.rp4$y)

(f.rp5 <- rpart(Diet ~ ., data=tree.data, minsplit = 8, cp = .0001))
prp(f.rp5, type=1, extra=3)

fancyRpartPlot(f.rp2)
fancyRpartPlot(f.rp3)
fancyRpartPlot(f.rp4)
fancyRpartPlot(f.rp5)

## For Cp-plot, we want a more "interesting" (larger) example
plotcp(f.rp2) 
plotcp(f.rp3) 
plotcp(f.rp4)

In [ ]:
# silly too large model
f.rp.age <- rpart(Diet ~ ., data = tree.data,
               control = rpart.control(minsplit = 5, cp=0))

tree.data <- metab_data4[,c(8,11,12:ncol(metab_data4))]
tree.data <- filter(tree.data, Age.Cohort %in% c(1:4))  %>% filter(Diet == "CD")
f.rp.age.0  <- rpart(Age.Cohort ~ ., data=tree.data)

prp(f.rp)

In [ ]:
## slightly larger than default:
(f.rp2.age.1 <- rpart(Age.Cohort ~ ., data=tree.data, minsplit = 4, cp = .1))
prp(f.rp2.age.1, type=3, extra=3)

(f.rp2.age.2 <- rpart(Age.Cohort ~ ., data=tree.data, minsplit = 4, cp = .01))
prp(f.rp2.age.2, type=3, extra=3)

(f.rp2.age.3 <- rpart(Age.Cohort ~ ., data=tree.data, minsplit = 4, cp = .001))
prp(f.rp2.age.3, type=3, extra=3)

In [ ]:
## For Cp-plot, we want a more "interesting" (larger) example
plotcp(f.rp) #
plotcp(f.rp2) 

In [ ]:
# PARTY package
```{r}
library(party)

frmla = Diet ~ .
 
(ct = ctree(frmla, data = tree.data))
plot(ct, main="Conditional Inference Tree")
 
#Table of prediction errors
table(predict(ct), raw$Metal)
 
# Estimated class probabilities
tr.pred = predict(ct, newdata=raw, type="prob")

In [ ]:
# MAPTREE
```{r}
library(maptree)
library(cluster)
draw.tree( clip.rpart (rpart ( tree.data), best=7),
nodeinfo=TRUE, units="species",
cases="cells", digits=0)
a = agnes ( raw[2:4], method="ward" )
names(a)
a$diss
b = kgs (a, a$diss, maxclust=20)
 
plot(names(b), b, xlab="# clusters", ylab="penalty", type="n")
xloc = names(b)[b==min(b)]
yloc = min(b)
ngon(c(xloc,yloc+.75,10, "dark green"), angle=180, n=3)
apply(cbind(names(b), b, 3, 'blue'), 1, ngon, 4) # cbind(x,y,size,color)
```



```





```



```




In [ ]:
## EVTREE (Evoluationary Learning)
```{r}
library(evtree)
 
ev.raw = evtree(frmla, data=tree.data)
plot(ev.raw)
table(predict(ev.raw), raw$Metal)
1-mean(predict(ev.raw) == raw$Metal)

In [ ]:
## randomForest
```{r}
library(randomForest)
tree.data[1] <- droplevels(tree.data[1])

tree.data.train <- tree.data[1:nrow(tree.data)/2,]
tree.data.test <- tree.data[ (nrow(tree.data)/2 + 1) : nrow(tree.data),]

fit.rf = randomForest(data=tree.data.train, Diet ~ ., ntree=1000, proximity=TRUE)
print(fit.rf)
importance(fit.rf)
plot(fit.rf)

varImpPlot(fit.rf)



plot( importance(fit.rf), lty=2, pch=16)
lines(importance(fit.rf))
imp = importance(fit.rf)
impvar = rownames(imp)[order(imp[, 1], decreasing=TRUE)]
op = par(mfrow=c(1, 3))

for (i in seq_along(impvar)) {
partialPlot(fit.rf, tree.data, impvar[i], xlab=impvar[i],
main=paste("Partial Dependence on", impvar[i]),
ylim=c(0, 1))
}


Pred.rf <-predict(fit.rf,newdata=tree.data.test)
table(Pred.rf, tree.data.test$Diet)

plot(randomForest::margin(fit.rf, tree.data.test$Diet))


tune.rf <- tuneRF(tree.data.train[,-5],tree.data.train[,5], stepFactor=0.5)
print(tune.rf)

In [ ]:
## CORElearn
```{r} 
library(CORElearn)
## Random Forests
fit.rand.forest = CoreModel(frmla, data=tree.data, model="rf", selectionEstimator="MDL", minNodeWeightRF=5, rfNoTrees=100)
plot(fit.rand.forest)
 
## decision tree with naive Bayes in the leaves
fit.dt = CoreModel(frmla, tree.data, model="tree", modelType=4)
plot(fit.dt, tree.data)
 
fit.rt = CoreModel(Diet~., tree.data, model="regTree", modelTypeReg=1)
summary(fit.rt)

plot(fit.rt, tree.data, graphType="prototypes")
 
pred = predict(fit.dt, tree.data)
print(pred)
plot(pred$class)

In [ ]:
# Logisitical Regression
```{r}

logreg.data.train <- tree.data[1:nrow(tree.data)/2,]
logreg.data.test <- tree.data[ (nrow(tree.data)/2 + 1) : nrow(tree.data),]

model <- glm(Diet ~.,family=binomial(link='logit'),data=logreg.data.train[1:10])
model <- glm(Diet ~.,logreg.data.train)

summary(model)

anova(model, test="Chisq")

library(pscl)
pR2(model)

fitted.results <- predict(model,newdata=logreg.data.test[2:10],type='response')
fitted.results <- ifelse(fitted.results > 0.5,1,0)

misClasificError <- mean(fitted.results != logreg.data.test$Diet)
print(paste('Accuracy',1-misClasificError))


library(ROCR)
p <- predict(model, newdata=logreg.data.test, type="response")
pr <- prediction(p, logreg.data.test$Diet)
prf <- performance(pr, measure = "tpr", x.measure = "fpr")
plot(prf)


auc <- performance(pr, measure = "auc")
auc <- auc@y.values[[1]]
auc
```
